# 进阶篇：pandas操作技巧系列（一）

## 操作技巧1：变量类型自动转换
在用pandas进行数据清洗的过程中，变量的类型转换是一个必然会遇到的步骤。清洗初期查看dtypes经常出现object类型，但其实变量本身可能就是个字符串，或者是数字（但因存在空值，导致出现了object类型）。

通常大家所熟知的方法是使用astype进行类型转换，或者自己利用astype造个轮子，写个函数方法实现自动转换类型。

本次东哥介绍一个pandas里可实现自动转换变量类型的方法convert_dtypes。利用它可以一次性全部转换为最理想的类型。

### 使用方法
默认情况下，convert_dtypes将尝试将Series或DataFrame中的每个Series转换为支持的dtypes。它可以对Series和DataFrame都直接使用。

这个方法的参数如下。
```python
# 是否应将对象dtypes转换为最佳类型
infer_objects bool，默认为True

# 对象dtype是否应转换为StringDtype()
convert_string bool，默认为True

# 如果可能，是否可以转换为整数扩展类型
convert_integer bool，默认为True

# 对象dtype是否应转换为BooleanDtypes()
convert_boolean bool，默认为True

# 如果可能，是否可以转换为浮动扩展类型。
# 如果convert_integer也为True，则如果可以将浮点数忠实地转换为整数，则将优先考虑整数dtype
convert_floating bool，默认为True
```

### 实例
下面看个例子。

首先创建一组数据，通过dtype规定每个变量的类型。

In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.DataFrame(
    {
        "a": pd.Series([1, 2, 3], dtype=np.dtype("int32")),
        "b": pd.Series(["x", "y", "z"], dtype=np.dtype("O")),
        "c": pd.Series([True, False, np.nan], dtype=np.dtype("O")),
        "d": pd.Series(["h", "i", np.nan], dtype=np.dtype("O")),
        "e": pd.Series([10, np.nan, 20], dtype=np.dtype("float")),
        "f": pd.Series([np.nan, 100.5, 200], dtype=np.dtype("float")),
    }
)

#### DataFrame 变量类型转换

In [5]:
df

,a,b,c,d,e,f
0,1,x,True,h,10.0,NaN
1,2,y,False,i,NaN,100.5
2,3,z,NaN,NaN,20.0,200.0


In [6]:
df.dtypes

a      int32
b     object
c     object
d     object
e    float64
f    float64
dtype: object

通过结果可以看到，变量都是是创建时默认的类型。但其实变量是有整数、字符串、布尔的，其中有的还存在空值。

In [7]:
dfn = df.convert_dtypes()
dfn

,a,b,c,d,e,f
0,1,x,True,h,10,<NA>
1,2,y,False,i,<NA>,100.5
2,3,z,<NA>,<NA>,20,200.0


下面使用convert_dtypes进行转换。

In [8]:
dfn.dtypes

a      Int32
b     string
c    boolean
d     string
e      Int64
f    Float64
dtype: object

变量类型已经转换为我们想要的了。

#### Series 变量类型转换
对Series的转换也是一样的。下面的Seires中由于存在nan空值所以类型为object。

In [9]:
s = pd.Series(["a", "b", np.nan])
s

0      a
1      b
2    NaN
dtype: object

然后我们通过convert_dtypes成功转换为String。

In [10]:
s.convert_dtypes()

0       a
1       b
2    <NA>
dtype: string

如果未来增加了新类型，convert_dtypes方法也会同步更新，并支持新的变量类型。

## 操作技巧2：json数据解析

用API和文档数据库会返回嵌套的JSON对象，当我们使用Python尝试将嵌套结构中的键转换为列时，数据加载到pandas中往往会得到如下结果：

> df = pd.DataFrame.from_records（results [“ issues”]，columns = [“ key”，“ fields”]）

![](https://camo.githubusercontent.com/348a155256bc6ae9e19dbb0a8d7da62be2d4ab6e49b9c6b4bbb5c895767760c7/68747470733a2f2f6d6d62697a2e717069632e636e2f737a5f6d6d62697a5f706e672f4e4f4d35484e326963587a787571636b5547667646347a4751345a31466f664b5747654f3256427a4a564a6f754b42524966326c4e576655314d316963597a78377a584c4e4474616961744a5953384166486962547836696146512f3634303f77785f666d743d706e672674703d7765627026777866726f6d3d352677785f6c617a793d312677785f636f3d31)

说明：这里results是一个大的字典，issues是results其中的一个键，issues的值为一个嵌套JSON对象字典的列表，后面会看到JSON嵌套结构。

问题在于API返回了嵌套的JSON结构，而我们关心的键在对象中确处于不同级别。

嵌套的JSON结构张成这样的。
![](https://camo.githubusercontent.com/ff92a9d6b9d8f01007eb116940b6091a924ded78a6ce7429726d3f6700a51066/68747470733a2f2f6d6d62697a2e717069632e636e2f737a5f6d6d62697a5f706e672f4e4f4d35484e326963587a787571636b5547667646347a4751345a31466f664b5747426d36644a57427a48644b664e456963696357343347496839477a346652314335485037535563784270364d696148493969624e6962425732772f3634303f77785f666d743d706e672674703d7765627026777866726f6d3d352677785f6c617a793d312677785f636f3d31)

而我们想要的是下面这样的。

![](https://mmbiz.qpic.cn/sz_mmbiz_png/NOM5HN2icXzxuqckUGfvF4zGQ4Z1FofKWGeO2VBzJVJouKBRIf2lNWfU1M1icYzx7zXLNDtaiatJYS8AfHibTx6iaFQ/640?wx_fmt=png&tp=webp&wxfrom=5&wx_lazy=1&wx_co=1)

下面以一个API返回的数据为例，API通常包含有关字段的元数据。假设下面这些是我们想要的字段。

- key：JSON密钥，在第一级的位置。
- summary：第二级的“字段”对象。
- status name：第三级位置。
- statusCategory name：位于第4个嵌套级别。

如上，我们选择要提取的字段在issues列表内的JSON结构中分别处于4个不同的嵌套级别，一环扣一环。
```json
{
  "expand": "schema,names",
  "issues": [
    {
      "fields": {
        "issuetype": {
          "avatarId": 10300,
          "description": "",
          "id": "10005",
          "name": "New Feature",
          "subtask": False
        },
        "status": {
          "description": "A resolution has been taken, and it is awaiting verification by reporter. From here issues are either reopened, or are closed.",
          "id": "5",
          "name": "Resolved",
          "statusCategory": {
            "colorName": "green",
            "id": 3,
            "key": "done",
            "name": "Done",
          }
        },
        "summary": "Recovered data collection Defraglar $MFT problem"
      },
      "id": "11861",
      "key": "CAE-160",
    },
    {
      "fields": { 
... more issues],
  "maxResults": 5,
  "startAt": 0,
  "total": 160
}
```

### 解决方案
pandas中有一个牛逼的内置功能叫 .json_normalize。

pandas的文档中提到：将半结构化JSON数据规范化为平面表。

前面方案的所有代码，用这个内置功能仅需要3行就可搞定。步骤很简单，懂了下面几个用法即可。

确定我们要想的字段，使用 . 符号连接嵌套对象。

将想要处理的嵌套列表（这里是results["issues"]）作为参数放进 .json_normalize 中。

过滤我们定义的FIELDS列表。
```python
FIELDS = ["key", "fields.summary", "fields.issuetype.name", "fields.status.name", "fields.status.statusCategory.name"]
df = pd.json_normalize(results["issues"])
df[FIELDS]
```
![](https://camo.githubusercontent.com/557bc7e474fb9eee2d9b71c6f5899f07deca1f0698d961042d7751a9453cbb02/68747470733a2f2f6d6d62697a2e717069632e636e2f737a5f6d6d62697a5f706e672f4e4f4d35484e326963587a7875536f787841626b77566d7574704356754f4d43486c6962624e3351336563725458727a39764c69623476616c41456141537a345a76424669624c654244626b6a65436e494a47745533363170412f3634303f77785f666d743d706e672674703d7765627026777866726f6d3d352677785f6c617a793d312677785f636f3d31)

### 其它操作
#### 记录路径
除了像上面那样传递results["issues"]列表之外，我们还使用record_path参数在JSON对象中指定列表的路径。

```oython
# 使用路径而不是直接用results["issues"]
pd.json_normalize(results, record_path="issues")[FIELDS]
```

#### 自定义分隔符
还可以使用sep参数自定义嵌套结构连接的分隔符，比如下面将默认的“.”替换“-”。

```python
### 用 "-" 替换默认的 "."
FIELDS = ["key", "fields-summary", "fields-issuetype-name", "fields-status-name", "fields-status-statusCategory-name"]
pd.json_normalize(results["issues"], sep = "-")[FIELDS]
```

#### 控制递归
如果不想递归到每个子对象，可以使用max_level参数控制深度。在这种情况下，由于statusCategory.name字段位于JSON对象的第4级，因此不会包含在结果DataFrame中。
```python
#只深入到嵌套第二级 
pd.json_normalize(results, record_path="issues", max_level = 2)
```

## 操作技巧3：Datetime时间类型提速
平时我们运行pandas少不了和时间打交道，而大多情况下许多朋友都是暴力解决问题，直接让pandas自己转换和处理。

对于平时的学习和小测试是没什么问题的，但当跑一些大数据的时候往往会非常的慢，而这个时间性能其实是完全可以优化的。

本次东哥介绍一个非常简单的操作，使用Datetime变换时间类型，让你的代码运行速度飞速提升。

下面，我们来看一个例子。

In [4]:
df = pd.read_csv('data/demand_profile.csv')
df.head()

,date_time,energy_kwh
0,1/1/13 0:00,0.586
1,1/1/13 1:00,0.580
2,1/1/13 2:00,0.572
3,1/1/13 3:00,0.596
4,1/1/13 4:00,0.592


从运行上面代码得到的结果来看，好像没有什么问题。但实际上pandas和numpy都有一个 dtypes 的概念。如果没有特殊声明，那么date_time将会使用一个 object 的 dtype 类型，如下面代码所示：

In [13]:
df.dtypes

date_time      object
energy_kwh    float64
dtype: object

In [14]:
type(df.iat[0, 0])

str

object 类型像一个大的容器，不仅仅可以承载 str，也可以包含那些不能很好地融进一个数据类型的任何特征列。而如果我们将日期作为 str 类型就会极大的影响效率。

因此，对于时间序列的数据而言，我们需要让上面的date_time列格式化为datetime对象数组（pandas称之为时间戳）。pandas在这里操作非常简单，操作如下：

In [15]:
df['date_time'] = pd.to_datetime(df['date_time'])
df['date_time'].dtype

dtype('<M8[ns]')

我们来运行一下这个df看看转化后的效果是什么样的。

In [16]:
df.head()

,date_time,energy_kwh
0,2013-01-01 00:00:00,0.586
1,2013-01-01 01:00:00,0.580
2,2013-01-01 02:00:00,0.572
3,2013-01-01 03:00:00,0.596
4,2013-01-01 04:00:00,0.592


date_time的格式已经自动转化了，但这还没完，在这个基础上，我们还是可以继续提高运行速度的。如何提速呢？为了更好的对比，我们首先通过 timeit 装饰器来测试一下上面代码的转化时间。

In [1]:
from zyf_timer import repeat_timeit

In [5]:
@repeat_timeit(repeat=3, number=10)
def convert(df, column_name):
    return pd.to_datetime(df[column_name])

df['date_time'] = convert(df, 'date_time')

Function convert -> 3 trials 10 function calls: average trial 11.169 seconds, average function call 1.117 seconds


1.117s，看上去挺快，但其实可以更快，我们来看一下下面的方法。

In [10]:
@repeat_timeit(repeat=3, number=100)
def convert_with_format(df, column_name):
    return pd.to_datetime(df[column_name],format='%d/%m/%y %H:%M')

df['date_time'] = convert(df, 'date_time')

Function convert -> 3 trials 10 function calls: average trial 0.158 seconds, average function call 0.016 seconds


结果只有0.016s，快了将近70倍。

原因是：我们设置了转化的格式format。由于在CSV中的datetimes并不是 ISO 8601 格式的，如果不进行设置的话，那么pandas将使用 dateutil 包把每个字符串str转化成date日期。

相反，如果原始数据datetime已经是 ISO 8601 格式了，那么pandas就可以立即使用最快速的方法来解析日期。这也就是为什么提前设置好格式format可以提升这么多。

当然，这个只是在时间处理上的一个提速小操作。